In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import torch.optim as optim
import math

In [2]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        nb_hidden = 200
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(5*5 * 64, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2))
        x = F.relu(self.fc1(x.view(-1, 5*5 * 64)))
        x = self.fc2(x)
        # print("x = ",x.size(), x)
        return x

In [3]:
#2 
def train_model(model, train_input, train_target, mini_batch_size):
    # We do this with mini-batches
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=η, momentum=0.3)
    # print("Size = ", train_input.size(), train_target.size())
    acc_loss = 0
    for b in range(0, train_input.size(0), mini_batch_size):
        #--------------
          # torch.narrow(input, dim, start, length) → Tensor
        #--------------
        # print("train_input.narrow",train_input.narrow(0, b, mini_batch_size).size())
        output = model(train_input.narrow(0, b, mini_batch_size))#表示取出train_input中第0维上索引从b开始到index+mini_batch_size-1的所有元素
        loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
        acc_loss = acc_loss + loss.item()
        model.zero_grad()
        loss.backward()
        optimizer.step()

    return model, acc_loss 

In [23]:
import dlc_practical_prologue as prologue
N = 1000
data = prologue.generate_pair_sets(N)
train_data = data[:3]
test_data = data[3:]

train_input, train_target, train_classes = train_data[0], train_data[1], train_data[2]
test_input, test_target, test_classes = test_data[0], test_data[1], test_data[2]
print(train_input.size(), train_target.size(), train_classes.size())

train_input_1 =  train_input[:,0].unsqueeze(1)
train_input_2 =  train_input[:,1].unsqueeze(1)
train_classes_1 =  train_classes[:,0]
train_classes_2 =  train_classes[:,1]
train_target_2 =  train_classes[:,1]
test_input_1 =  test_input[:,0].unsqueeze(1)
test_input_2 =  test_input[:,1].unsqueeze(1)
test_classes_1 =  test_classes[:,0]
test_classes_2 =  test_classes[:,1]
print(test_input_1.size(), test_classes_1.size(),test_classes_1[1])

torch.Size([1000, 2, 14, 14]) torch.Size([1000]) torch.Size([1000, 2])
torch.Size([1000, 1, 14, 14]) torch.Size([1000]) tensor(4)


In [66]:
def compute_nb_errors(model, data_input, data_target, mini_batch_size):
  error = 0
  output_class = torch.ones(data_input.size(0))
  for b in range(0, data_input.size(0), mini_batch_size): 
    output = model(data_input.narrow(0, b, mini_batch_size))
    
    for i in range(mini_batch_size): 
      output_class[b+i] = torch.argmax(output[i])
      if output_class[b+i] != data_target[b+i]:
        error += 1
  acc = 1 - (error/data_input.size(0))
  return acc, output_class

In [67]:
model = Net()
η = 0.002 #If it too big, it will fail!
mini_batch_size = 50
nb_epochs =30
#print(train_input, train_target)
for e in range(nb_epochs):
  
    #print(train_input_1, train_classes_1)
    model, acc_loss = train_model(model, train_input_1, train_classes_1, mini_batch_size)
    acc, output_class_1 = compute_nb_errors(model, test_input_1, test_classes_1, mini_batch_size)
    print('epoch :', e, ' loss :', round(acc_loss))
    print('acc : ', acc)
    print('----------')

epoch : 0  loss : 106
acc :  0.772
----------
epoch : 1  loss : 13
acc :  0.885
----------
epoch : 2  loss : 7
acc :  0.91
----------
epoch : 3  loss : 5
acc :  0.912
----------
epoch : 4  loss : 4
acc :  0.917
----------
epoch : 5  loss : 3
acc :  0.922
----------
epoch : 6  loss : 2
acc :  0.927
----------
epoch : 7  loss : 2
acc :  0.929
----------
epoch : 8  loss : 1
acc :  0.9319999999999999
----------
epoch : 9  loss : 1
acc :  0.9319999999999999
----------
epoch : 10  loss : 1
acc :  0.937
----------
epoch : 11  loss : 1
acc :  0.937
----------
epoch : 12  loss : 1
acc :  0.937
----------
epoch : 13  loss : 1
acc :  0.938
----------
epoch : 14  loss : 0
acc :  0.94
----------
epoch : 15  loss : 0
acc :  0.94
----------
epoch : 16  loss : 0
acc :  0.94
----------
epoch : 17  loss : 0
acc :  0.9410000000000001
----------
epoch : 18  loss : 0
acc :  0.943
----------
epoch : 19  loss : 0
acc :  0.944
----------
epoch : 20  loss : 0
acc :  0.944
----------
epoch : 21  loss : 0
acc : 

In [68]:
for e in range(nb_epochs):
    model, acc_loss = train_model(model, train_input_2, train_classes_2, mini_batch_size)
    acc, output_class_2 = compute_nb_errors(model, test_input_2, test_classes_2, mini_batch_size)
    print('epoch :', e, ' loss :', round(acc_loss))
    print('acc : ', acc)
    print('----------')

epoch : 0  loss : 7
acc :  0.938
----------
epoch : 1  loss : 4
acc :  0.944
----------
epoch : 2  loss : 2
acc :  0.944
----------
epoch : 3  loss : 2
acc :  0.947
----------
epoch : 4  loss : 1
acc :  0.945
----------
epoch : 5  loss : 1
acc :  0.944
----------
epoch : 6  loss : 1
acc :  0.944
----------
epoch : 7  loss : 1
acc :  0.945
----------
epoch : 8  loss : 0
acc :  0.948
----------
epoch : 9  loss : 0
acc :  0.947
----------
epoch : 10  loss : 0
acc :  0.948
----------
epoch : 11  loss : 0
acc :  0.949
----------
epoch : 12  loss : 0
acc :  0.95
----------
epoch : 13  loss : 0
acc :  0.951
----------
epoch : 14  loss : 0
acc :  0.952
----------
epoch : 15  loss : 0
acc :  0.951
----------
epoch : 16  loss : 0
acc :  0.949
----------
epoch : 17  loss : 0
acc :  0.949
----------
epoch : 18  loss : 0
acc :  0.949
----------
epoch : 19  loss : 0
acc :  0.95
----------
epoch : 20  loss : 0
acc :  0.95
----------
epoch : 21  loss : 0
acc :  0.95
----------
epoch : 22  loss : 0
acc

In [ ]:
def compute_num_errors(data_input1, data_input2, data_target, mini_batch_size):
  error = 0
  for b in range(0, data_input.size(0), mini_batch_size): 
    for i in range(mini_batch_size): 
      
      if torch.argmax(output[i]) != data_target[b+i]:
        error += 1
  acc = 1 - (error/data_input.size(0))
  return acc

In [87]:
def predict_target(data_output1, data_output2):
  output = torch.ones(len(data_output1))
  for i in range(len(data_output1)):
    #print(data_output1[i])
    if data_output1[i].item() > data_output2[i].item():
        output[i] = 0
    else:
        output[i] = 1
  return output

In [89]:
target_output = predict_target(output_class_1, output_class_2)
print(target_output, target_output.size())

tensor([1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1.,
        1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1.,
        0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0.,
        1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0.,
        0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1.,
        0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1.,
        1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1.,
        0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0.,
        0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
        1., 0., 0., 0., 1., 1., 0., 1., 

In [71]:
for e in range(nb_epochs):
    model, acc_loss = train_model(model, train_input_2, train_classes_2, mini_batch_size)
    acc = compute_nb_errors(model, test_input_2, test_classes_2, mini_batch_size)
    print('epoch :', e, ' loss :', round(acc_loss))
    print('acc : ', acc)
    print('----------')

epoch : 0  loss : 0
acc :  (0.951, tensor([6., 2., 6., 7., 6., 6., 7., 8., 2., 6., 9., 9., 3., 4., 7., 1., 0., 1.,
        3., 4., 1., 1., 2., 2., 8., 8., 3., 5., 9., 2., 0., 7., 2., 3., 6., 2.,
        6., 1., 4., 7., 9., 4., 7., 7., 4., 8., 8., 4., 9., 4., 1., 5., 0., 1.,
        5., 1., 8., 7., 1., 9., 8., 5., 6., 4., 4., 2., 4., 1., 2., 8., 3., 3.,
        6., 1., 4., 7., 1., 5., 8., 8., 4., 1., 1., 3., 6., 9., 8., 4., 7., 0.,
        3., 5., 4., 8., 0., 9., 7., 4., 1., 2., 3., 7., 7., 3., 8., 9., 0., 7.,
        2., 2., 7., 5., 9., 2., 4., 9., 6., 5., 1., 5., 9., 9., 6., 3., 7., 6.,
        3., 0., 2., 4., 0., 8., 0., 4., 7., 4., 3., 1., 3., 1., 5., 5., 8., 7.,
        8., 7., 9., 8., 3., 2., 3., 6., 3., 6., 8., 2., 6., 1., 1., 8., 1., 9.,
        0., 3., 9., 4., 4., 5., 1., 0., 3., 7., 7., 1., 6., 7., 6., 4., 8., 3.,
        3., 7., 1., 2., 8., 0., 1., 5., 3., 7., 1., 8., 7., 2., 4., 5., 8., 5.,
        2., 7., 7., 1., 1., 7., 3., 9., 7., 3., 9., 0., 0., 8., 6., 6., 1., 2.,
     

In [72]:
model1 = Net()
model2 = Net()
η = 0.2
mini_batch_size = 2
nb_epochs = 25
#print(train_input, train_target)
for e in range(nb_epochs):
    print("Data = ", train_data[0].size(), train_data[2].size())
    model, acc_loss = train_model(model1, train_data[0][0], train_data[2][0], mini_batch_size)

    acc = compute_nb_errors(model, test_input, test_target)
    print('epoch :', e, ' loss :', round(acc_loss, 3))
    print('acc : ', acc)
    print('----------')

for e in range(nb_epochs):
    model, acc_loss = train_model(model2, train_data[0][1], train_data[2][1], mini_batch_size)

    acc = compute_nb_errors(model, test_input, test_target)
    print('epoch :', e, ' loss :', round(acc_loss, 3))
    print('acc : ', acc)
    print('----------')

Data =  torch.Size([1000, 2, 14, 14]) torch.Size([1000, 2])


RuntimeError: ignored

In [ ]:
class Net2(nn.Module):
    def __init__(self):
        super().__init__()
        nb_hidden = 200
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(2*32, 2*32, kernel_size=5)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=2)
        self.fc1 = nn.Linear(9 * 64, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 10)

    def forward(self, x):
        x1 = x[0]
        print("x1= ", x1)
        x1 = F.relu(F.max_pool2d(self.conv1(x1), kernel_size=2))
        x1 = F.relu(F.max_pool2d(self.conv2(x1), kernel_size=2))
        x1 = F.relu(self.conv3(x1))
        x1 = F.relu(self.fc1(x1.view(-1, 9 * 64)))
        x1 = self.fc2(x1)
        x2 = x[1]
        x2 = F.relu(F.max_pool2d(self.conv1(x2), kernel_size=2))
        x2 = F.relu(F.max_pool2d(self.conv2(x2), kernel_size=2))
        x2 = F.relu(self.conv3(x2))
        x2 = F.relu(self.fc1(x2.view(-1, 9 * 64)))
        x2 = self.fc2(x2)
        y = [x1, x2]
        return y